# 深度学习时间序列预测

**LSTM 与 Transformer 在资金费率预测中的应用**

---

## 目录

### 第一部分：基础知识
1. 时间序列预测简介
2. 深度学习在金融预测中的应用
3. LSTM 与 Transformer 架构概述

### 第二部分：LSTM 模型
4. LSTM 原理详解
5. 数据准备与特征工程
6. LSTM 模型构建（PyTorch）
7. 模型训练与验证
8. 资金费率预测实战

### 第三部分：Transformer 模型
9. Transformer 架构详解
10. 时间序列 Transformer 实现
11. Transformer 资金费率预测

### 第四部分：模型对比与优化
12. LSTM vs Transformer 性能对比
13. 超参数调优
14. 实战建议与总结

---

# 第一部分：基础知识

---

## 1. 时间序列预测简介

### 1.1 什么是时间序列预测？

**时间序列预测** 是利用历史数据来预测未来值的方法。在金融领域，它被广泛用于：
- 价格预测
- 波动率预测
- 资金费率预测
- 交易量预测

### 1.2 时间序列的特点

| 特性 | 描述 | 资金费率示例 |
|------|------|-------------|
| **趋势（Trend）** | 长期上升或下降趋势 | 牛市期间费率整体偏高 |
| **季节性（Seasonality）** | 周期性重复模式 | 结算时间点的规律性 |
| **周期性（Cyclicality）** | 非固定周期的波动 | 市场情绪周期 |
| **随机性（Noise）** | 无法解释的波动 | 突发事件影响 |

### 1.3 传统方法 vs 深度学习

```
┌─────────────────────────────────────────────────────────────────┐
│                      预测方法对比                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   传统统计方法                    深度学习方法                  │
│   ├── ARIMA                      ├── RNN/LSTM                  │
│   ├── GARCH                      ├── GRU                       │
│   ├── Prophet                    ├── Transformer               │
│   └── VAR                        └── TCN                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## 2. 深度学习在金融预测中的应用

### 2.1 为什么选择深度学习？

金融时间序列具有以下特点：
1. **非线性关系**：价格变动与影响因素之间存在复杂的非线性关系
2. **长期依赖**：当前价格可能受到很久之前事件的影响
3. **多变量交互**：多个因素相互影响
4. **噪声干扰**：信号与噪声混合

## 3. LSTM 与 Transformer 架构概述

### 3.1 LSTM（Long Short-Term Memory）

LSTM 是一种特殊的 RNN，通过门控机制解决梯度消失问题：

```
┌─────────────────────────────────────────────────────────────────┐
│                        LSTM 单元结构                            │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   三个门的作用：                                               │
│   • 遗忘门：决定丢弃哪些旧信息                                 │
│   • 输入门：决定存储哪些新信息                                 │
│   • 输出门：决定输出哪些信息                                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 3.2 Transformer

Transformer 使用自注意力机制，可以并行处理序列。

### 3.3 LSTM vs Transformer 对比

| 特性 | LSTM | Transformer |
|------|------|-------------|
| **处理方式** | 顺序处理 | 并行处理 |
| **长期依赖** | 通过门控机制 | 通过注意力机制 |
| **计算复杂度** | O(n) | O(n²) |
| **训练速度** | 较慢 | 较快 |

---

# 第二部分：LSTM 模型

---

## 4. LSTM 原理详解

### 4.1 LSTM 数学公式

**遗忘门（Forget Gate）**：
$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

**输入门（Input Gate）**：
$$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$

**细胞状态更新**：
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$$

**输出门（Output Gate）**：
$$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$
$$h_t = o_t \odot \tanh(C_t)$$

## 5. 数据准备与特征工程

In [ ]:
# 5.1 导入依赖库

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ccxt
from datetime import datetime, timedelta
import warnings

# 深度学习库
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# 检查 GPU 可用性
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✅ 使用设备: {device}')
print(f'✅ PyTorch 版本: {torch.__version__}')

In [ ]:
# 5.2 获取资金费率数据

def get_funding_rate_history(symbol: str, days: int = 180) -> pd.DataFrame:
    """
    获取历史资金费率
    """
    exchange = ccxt.binance({
        'enableRateLimit': True,
        'options': {'defaultType': 'swap'}
    })
    
    since = int((datetime.now() - timedelta(days=days)).timestamp() * 1000)
    
    all_rates = []
    while True:
        rates = exchange.fetch_funding_rate_history(symbol, since=since, limit=1000)
        if not rates:
            break
        all_rates.extend(rates)
        since = rates[-1]['timestamp'] + 1
        if len(rates) < 1000:
            break
    
    df = pd.DataFrame(all_rates)
    if not df.empty:
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.set_index('datetime')
    
    return df

# 获取 BTC 资金费率数据
print('📊 获取 BTC 资金费率数据...')
df_funding = get_funding_rate_history('BTC/USDT:USDT', days=365)
print(f'✅ 获取到 {len(df_funding)} 条记录')

In [ ]:
# 5.3 数据预处理与特征工程

def create_features(df: pd.DataFrame, target_col: str = 'fundingRate') -> pd.DataFrame:
    """
    创建时间序列特征
    """
    df_feat = df[[target_col]].copy()
    
    # 滞后特征
    for i in range(1, 25):
        df_feat[f'lag_{i}'] = df_feat[target_col].shift(i)
    
    # 滚动统计
    for window in [3, 6, 12, 24]:
        df_feat[f'rolling_mean_{window}'] = df_feat[target_col].shift(1).rolling(window).mean()
        df_feat[f'rolling_std_{window}'] = df_feat[target_col].shift(1).rolling(window).std()
    
    # 时间特征
    df_feat['hour'] = df_feat.index.hour / 24
    df_feat['day_of_week'] = df_feat.index.dayofweek / 7
    
    return df_feat.dropna()

df_features = create_features(df_funding)
print(f'✅ 创建特征完成，共 {len(df_features.columns)} 列')

In [ ]:
# 5.4 创建序列数据集

class FundingRateDataset(Dataset):
    def __init__(self, data: np.ndarray, seq_length: int = 24):
        self.data = data
        self.seq_length = seq_length
    
    def __len__(self):
        return len(self.data) - self.seq_length
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_length]
        y = self.data[idx + self.seq_length, 0]
        return torch.FloatTensor(x), torch.FloatTensor([y])


def prepare_data(df: pd.DataFrame, seq_length: int = 24, train_ratio: float = 0.8):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(df.values)
    
    train_size = int(len(data_scaled) * train_ratio)
    train_data = data_scaled[:train_size]
    test_data = data_scaled[train_size:]
    
    train_dataset = FundingRateDataset(train_data, seq_length)
    test_dataset = FundingRateDataset(test_data, seq_length)
    
    return train_dataset, test_dataset, scaler

SEQ_LENGTH = 24
BATCH_SIZE = 32

train_dataset, test_dataset, scaler = prepare_data(df_features, SEQ_LENGTH)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f'✅ 数据准备完成')

## 6. LSTM 模型构建

In [ ]:
# 6.1 LSTM 模型定义

class LSTMPredictor(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int = 64, num_layers: int = 2, 
                 output_dim: int = 1, dropout: float = 0.2):
        super(LSTMPredictor, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        self.fc = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, output_dim)
        )
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        out = self.fc(out)
        return out

INPUT_DIM = df_features.shape[1]
lstm_model = LSTMPredictor(input_dim=INPUT_DIM, hidden_dim=64, num_layers=2).to(device)
print(lstm_model)
print(f'\n✅ 模型参数量: {sum(p.numel() for p in lstm_model.parameters()):,}')

## 7. 模型训练与验证

In [ ]:
# 7.1 训练函数

def train_model(model, train_loader, test_loader, epochs=100, lr=0.001, patience=10):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    train_losses, test_losses = [], []
    best_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                y_pred = model(X_batch)
                test_loss += criterion(y_pred, y_batch).item()
        
        test_loss /= len(test_loader)
        test_losses.append(test_loss)
        scheduler.step(test_loss)
        
        if test_loss < best_loss:
            best_loss = test_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}')
        
        if patience_counter >= patience:
            print(f'早停于 Epoch {epoch+1}')
            model.load_state_dict(best_model_state)
            break
    
    return train_losses, test_losses

In [ ]:
# 7.2 训练 LSTM 模型

print('📊 开始训练 LSTM 模型...')
train_losses, test_losses = train_model(lstm_model, train_loader, test_loader, epochs=100, lr=0.001, patience=15)
print('✅ 训练完成！')

In [ ]:
# 7.3 训练曲线可视化

fig = go.Figure()
fig.add_trace(go.Scatter(y=train_losses, mode='lines', name='训练损失', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=test_losses, mode='lines', name='测试损失', line=dict(color='red')))
fig.update_layout(title='LSTM 训练曲线', xaxis_title='Epoch', yaxis_title='Loss', height=400)
fig.show()

## 8. 资金费率预测实战

In [ ]:
# 8.1 模型评估

def evaluate_model(model, test_loader, scaler):
    model.eval()
    predictions, actuals = [], []
    
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_pred = model(X_batch)
            predictions.extend(y_pred.cpu().numpy())
            actuals.extend(y_batch.numpy())
    
    predictions = np.array(predictions).flatten()
    actuals = np.array(actuals).flatten()
    
    # 反标准化
    scale, mean = scaler.scale_[0], scaler.mean_[0]
    predictions = predictions * scale + mean
    actuals = actuals * scale + mean
    
    mae = mean_absolute_error(actuals, predictions)
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    r2 = r2_score(actuals, predictions)
    direction_correct = np.mean((predictions[1:] > predictions[:-1]) == (actuals[1:] > actuals[:-1]))
    
    return {'predictions': predictions, 'actuals': actuals, 'mae': mae, 'rmse': rmse, 'r2': r2, 'direction_accuracy': direction_correct}

lstm_results = evaluate_model(lstm_model, test_loader, scaler)
print(f'MAE: {lstm_results["mae"]*100:.6f}%')
print(f'RMSE: {lstm_results["rmse"]*100:.6f}%')
print(f'R²: {lstm_results["r2"]:.4f}')
print(f'方向准确率: {lstm_results["direction_accuracy"]*100:.2f}%')

---

# 第三部分：Transformer 模型

---

## 9. Transformer 架构详解

### 9.1 自注意力机制（Self-Attention）

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

### 9.2 多头注意力（Multi-Head Attention）

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O$$

### 9.3 位置编码（Positional Encoding）

$$PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)$$

## 10. 时间序列 Transformer 实现

In [ ]:
# 10.1 位置编码

class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, max_len: int = 5000, dropout: float = 0.1):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

In [ ]:
# 10.2 Transformer 预测模型

class TransformerPredictor(nn.Module):
    def __init__(self, input_dim: int, d_model: int = 64, nhead: int = 4, num_layers: int = 2,
                 dim_feedforward: int = 256, dropout: float = 0.1, output_dim: int = 1):
        super(TransformerPredictor, self).__init__()
        
        self.input_embedding = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout=dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc_out = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, output_dim)
        )
        self.d_model = d_model
    
    def forward(self, x):
        x = self.input_embedding(x) * np.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]
        return self.fc_out(x)

transformer_model = TransformerPredictor(input_dim=INPUT_DIM, d_model=64, nhead=4, num_layers=2).to(device)
print(transformer_model)
print(f'\n✅ 模型参数量: {sum(p.numel() for p in transformer_model.parameters()):,}')

## 11. Transformer 资金费率预测

In [ ]:
# 11.1 训练 Transformer 模型

print('📊 开始训练 Transformer 模型...')
transformer_train_losses, transformer_test_losses = train_model(
    transformer_model, train_loader, test_loader, epochs=100, lr=0.0005, patience=15
)
print('✅ 训练完成！')

In [ ]:
# 11.2 评估 Transformer 模型

transformer_results = evaluate_model(transformer_model, test_loader, scaler)
print(f'MAE: {transformer_results["mae"]*100:.6f}%')
print(f'RMSE: {transformer_results["rmse"]*100:.6f}%')
print(f'R²: {transformer_results["r2"]:.4f}')
print(f'方向准确率: {transformer_results["direction_accuracy"]*100:.2f}%')

---

# 第四部分：模型对比与优化

---

## 12. LSTM vs Transformer 性能对比

In [ ]:
# 12.1 模型对比

print('=' * 60)
print('📊 模型性能对比')
print('=' * 60)
print(f'{"指标":<15} {"LSTM":<20} {"Transformer":<20}')
print('-' * 60)
print(f'{"MAE (%)":<15} {lstm_results["mae"]*100:<20.6f} {transformer_results["mae"]*100:<20.6f}')
print(f'{"RMSE (%)":<15} {lstm_results["rmse"]*100:<20.6f} {transformer_results["rmse"]*100:<20.6f}')
print(f'{"R²":<15} {lstm_results["r2"]:<20.4f} {transformer_results["r2"]:<20.4f}')
print(f'{"方向准确率 (%)":<15} {lstm_results["direction_accuracy"]*100:<20.2f} {transformer_results["direction_accuracy"]*100:<20.2f}')
print('=' * 60)

In [ ]:
# 12.2 预测对比可视化

fig = go.Figure()
fig.add_trace(go.Scatter(y=lstm_results['actuals']*100, mode='lines', name='实际值', line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(y=lstm_results['predictions']*100, mode='lines', name='LSTM', line=dict(color='red', dash='dash')))
fig.add_trace(go.Scatter(y=transformer_results['predictions']*100, mode='lines', name='Transformer', line=dict(color='green', dash='dot')))
fig.update_layout(title='LSTM vs Transformer 预测对比', xaxis_title='时间步', yaxis_title='资金费率 (%)', height=500)
fig.show()

## 13. 超参数调优

### 常用超参数及建议范围

| 超参数 | LSTM 建议值 | Transformer 建议值 |
|--------|-------------|-------------------|
| hidden_dim/d_model | 32-128 | 32-128 |
| num_layers | 1-3 | 2-4 |
| dropout | 0.1-0.3 | 0.1-0.2 |
| learning_rate | 1e-3 - 1e-4 | 5e-4 - 1e-4 |
| batch_size | 16-64 | 16-64 |
| seq_length | 12-48 | 24-96 |

## 14. 实战建议与总结

### 14.1 模型选择建议

| 场景 | 推荐模型 | 原因 |
|------|----------|------|
| 短期预测 | LSTM | 更稳定，成本低 |
| 长期预测 | Transformer | 更好捕捉长期依赖 |
| 多变量预测 | Transformer | 注意力机制处理多维关系 |
| 资源受限 | GRU/LSTM | 更低的计算成本 |

### 14.2 常见陷阱

```
┌─────────────────────────────────────────────────────────────────┐
│                      常见陷阱与解决方案                         │
├─────────────────────────────────────────────────────────────────┤
│  1. 过拟合 → Dropout、正则化、早停                             │
│  2. 数据泄露 → 严格的时间分割                                  │
│  3. 分布漂移 → 定期重训练                                      │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
# 14.3 预测下一期

def predict_next(model, df_features, scaler, seq_length):
    model.eval()
    recent_data = df_features.iloc[-seq_length:].values
    recent_scaled = scaler.transform(recent_data)
    
    with torch.no_grad():
        x = torch.FloatTensor(recent_scaled).unsqueeze(0).to(device)
        pred = model(x).cpu().numpy()[0, 0]
    
    return pred * scaler.scale_[0] + scaler.mean_[0]

lstm_pred = predict_next(lstm_model, df_features, scaler, SEQ_LENGTH)
transformer_pred = predict_next(transformer_model, df_features, scaler, SEQ_LENGTH)

print('🔮 下一期资金费率预测')
print(f'LSTM: {lstm_pred*100:.4f}%  (年化: {lstm_pred*3*365*100:.2f}%)')
print(f'Transformer: {transformer_pred*100:.4f}%  (年化: {transformer_pred*3*365*100:.2f}%)')

---

## 总结

1. **LSTM**: 门控机制解决长期依赖，适合中短期预测
2. **Transformer**: 自注意力机制，并行计算效率高，适合长序列
3. **实战要点**: 特征工程、避免过拟合、模型选择与调优

### 下一步学习
- [ ] 多交易所套利策略
- [ ] 高频交易与订单簿分析
- [ ] 期权与永续合约组合策略